In [1]:
lines = '''start-A
start-b
A-c
A-b
b-d
A-end
b-end'''.splitlines()

lines='''dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc'''.splitlines()

lines = '''fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW'''.splitlines()

lines = '''start-A
start-b
A-c
A-b
b-d
A-end
b-end'''.splitlines()

#read from day12.txt then split lines
with open('day12.txt', 'r') as f:
    lines = f.read().splitlines()
    

In [2]:
# Fortunately, the sensors are still mostly working, and so you build a rough map of the remaining caves (your puzzle input). For example:

# start-A
# start-b
# A-c
# A-b
# b-d
# A-end
# b-end
# This is a list of how all of the caves are connected. You start in the cave named start, and your destination is the cave named end. An entry like b-d means that cave b is connected to cave d - that is, you can move between them.

#split by -
connectes = [line.split('-') for line in lines]

# So, the above cave system looks roughly like this:

#     start
#     /   \
# c--A-----b--d
#     \   /
#      end
# Your goal is to find the number of distinct paths that start at start, end at end, and don't visit small caves more than once. There are two types of caves: big caves (written in uppercase, like A) and small caves (written in lowercase, like b). It would be a waste of time to visit any small cave more than once, but big caves are large enough that it might be worth visiting them multiple times. So, all paths you find should visit small caves at most once, and can visit big caves any number of times.
def is_small(cave):
    if cave == 'start':return False
    return cave[0].islower()

#build a graph of caves
def build_graph(connectes):
    graph = {}
    caves= set()
    for connect in connectes:
        caves.add(connect[0])
        caves.add(connect[1])
        if connect[0] not in graph:
            graph[connect[0]] = []
        if connect[1] not in graph:
            graph[connect[1]] = []
        graph[connect[0]].append(connect[1])
        graph[connect[1]].append(connect[0])
    return caves, graph

caves, cave_system = build_graph(connectes)

#remove a small cave from cave_system
def remove_cave(small_cave, cave_system):
    for cave in cave_system:
        if small_cave in cave_system[cave]:
            cave_system[cave].remove(small_cave)
    return cave_system
#remove start
cave_system = remove_cave('start', cave_system)

#get all paths from start to end
end='end'
def get_paths(caves, cave_system, start='start',  path=[], small_caves=[]):
#   print(small_caves, path)
    path = path + [start]
    if start == end:
        return [path]
    small_caves = small_caves + [start] if is_small(start) else small_caves
    paths = []
    for node in cave_system[start]:
        if node not in small_caves:
            newpaths = get_paths(caves, cave_system, node, path, small_caves)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

paths = get_paths(caves, cave_system)
print(len(paths))

# Given these rules, there are 10 paths through this example cave system:

# start,A,b,A,c,A,end
# start,A,b,A,end
# start,A,b,end
# start,A,c,A,b,A,end
# start,A,c,A,b,end
# start,A,c,A,end
# start,A,end
# start,b,A,c,A,end
# start,b,A,end
# start,b,end
# (Each line in the above list corresponds to a single path; the caves visited by that path are listed in the order they are visited and separated by commas.)

# Note that in this cave system, cave d is never visited by any path: to do so, cave b would need to be visited twice (once on the way to cave d and a second time when returning from cave d), and since cave b is small, this is not allowed.

# Here is a slightly larger example:

# dc-end
# HN-start
# start-kj
# dc-start
# dc-HN
# LN-dc
# HN-end
# kj-sa
# kj-HN
# kj-dc
# The 19 paths through it are as follows:

# start,HN,dc,HN,end
# start,HN,dc,HN,kj,HN,end
# start,HN,dc,end
# start,HN,dc,kj,HN,end
# start,HN,end
# start,HN,kj,HN,dc,HN,end
# start,HN,kj,HN,dc,end
# start,HN,kj,HN,end
# start,HN,kj,dc,HN,end
# start,HN,kj,dc,end
# start,dc,HN,end
# start,dc,HN,kj,HN,end
# start,dc,end
# start,dc,kj,HN,end
# start,kj,HN,dc,HN,end
# start,kj,HN,dc,end
# start,kj,HN,end
# start,kj,dc,HN,end
# start,kj,dc,end
# Finally, this even larger example has 226 paths through it:

# fs-end
# he-DX
# fs-he
# start-DX
# pj-DX
# end-zg
# zg-sl
# zg-pj
# pj-he
# RW-he
# fs-DX
# pj-RW
# zg-RW
# start-pj
# he-WI
# zg-he
# pj-fs
# start-RW
# How many paths through this cave system are there that visit small caves at most once?


4413


In [3]:
# --- Part Two ---
# After reviewing the available paths, you realize you might have time to visit a single small cave twice. Specifically, big caves can be visited any number of times, a single small cave can be visited at most twice, and the remaining small caves can be visited at most once. However, the caves named start and end can only be visited exactly once each: once you leave the start cave, you may not return to it, and once you reach the end cave, the path must end immediately.
#get all paths from start to end with small caves visited at most twice
end='end'
def get_paths2(caves, cave_system, start='start', path=[], small_caves=[]):
    path = path + [start]
    if start == end:
        return 1
    if is_small(start): small_caves = small_caves + [start]
    small_caves.sort()
    hasTwice = False
    for i in range(len(small_caves)-1):
        if small_caves[i] == small_caves[i+1]:
            hasTwice = True
            break
    paths = 0
    for node in cave_system[start]:
        if (not is_small(node)) or (not hasTwice) or (node not in small_caves):
            newpaths = get_paths2(caves, cave_system, node, path, small_caves)
            paths += newpaths
    return paths

paths = get_paths2(caves, cave_system)
print(paths)


# Now, the 36 possible paths through the first example above are:

# start,A,b,A,b,A,c,A,end
# start,A,b,A,b,A,end
# start,A,b,A,b,end
# start,A,b,A,c,A,b,A,end
# start,A,b,A,c,A,b,end
# start,A,b,A,c,A,c,A,end
# start,A,b,A,c,A,end
# start,A,b,A,end
# start,A,b,d,b,A,c,A,end
# start,A,b,d,b,A,end
# start,A,b,d,b,end
# start,A,b,end
# start,A,c,A,b,A,b,A,end
# start,A,c,A,b,A,b,end
# start,A,c,A,b,A,c,A,end
# start,A,c,A,b,A,end
# start,A,c,A,b,d,b,A,end
# start,A,c,A,b,d,b,end
# start,A,c,A,b,end
# start,A,c,A,c,A,b,A,end
# start,A,c,A,c,A,b,end
# start,A,c,A,c,A,end
# start,A,c,A,end
# start,A,end
# start,b,A,b,A,c,A,end
# start,b,A,b,A,end
# start,b,A,b,end
# start,b,A,c,A,b,A,end
# start,b,A,c,A,b,end
# start,b,A,c,A,c,A,end
# start,b,A,c,A,end
# start,b,A,end
# start,b,d,b,A,c,A,end
# start,b,d,b,A,end
# start,b,d,b,end
# start,b,end
# The slightly larger example above now has 103 paths through it, and the even larger example now has 3509 paths through it.

# Given these new rules, how many paths through this cave system are there?


118803
